In [ ]:
import os
import pandas as pd
import numpy as np
import argparse
import ast
import matplotlib.pyplot as plt
# from together import Together
from alive_progress import alive_bar
from collections import Counter

from prompt import *
from utils import extract_option

PATH = "/home/cc/PHD/HealthBranches/"

In [ ]:
ds = pd.read_csv("/home/cc/PHD/HealthBranches/results/results_QUIZ_bench_deepseek-r1_8b.csv")
cnt = 0

for index, row in ds.iterrows():
    if extract_option(row['zero_shot']) not in ['A', 'B', 'C', 'D', 'E']:
        cnt +=1
        print(row['zero_shot'])
print(cnt)

In [ ]:
df = pd.read_csv("/home/cc/PHD/HealthBranches/results/results_QUIZ_bench_qwen2.5_7b.csv")

In [ ]:
# 2. Compute path length as the number of '->' in each string
df['path_len'] = df['path'].str.count(r'->')

# 3. Group by that new column
grouped = df.groupby('path_len')

# 4a. If you just want to see group sizes:
print("Rows per path-length:")
print(grouped.size().sort_index())

# 4b. If you want to iterate through each group:
for length, group in grouped:
    print(f"\n=== Path length = {length} (i.e. {length} '->' separators) ===")
    print(group[['path']])

In [ ]:
# 2. Compute path length (number of '->')
df['path_len'] = df['path'].str.count(r'->')

# 3. Apply extract_option to 'answer' and compare to 'real'
#    (Assuming extract_option is already defined or imported)
df['pred'] = df['zero_shot'].apply(extract_option)
df['correct'] = df['pred'] == df['real']

# 4. Group by path length and count matches & totals
agg = (
    df
    .groupby('path_len')
    .agg(
        total_rows=('correct','size'),
        correct_count=('correct','sum')
    )
    .sort_index()
    .reset_index()
)

total_rows_all = len(df)
agg['accuracy'] = agg['correct_count'] / total_rows_all

# 7) Plot bar chart of weighted accuracies
labels = [str(length) for length in agg.index]
weighted_accuracies = agg['accuracy']

plt.figure(figsize=(12, 8))
bars = plt.bar(labels, weighted_accuracies)
plt.xlabel('Path Length (number of "->" separators)')
plt.ylabel('Weighted Accuracy\n(corrects ÷ total rows)')
plt.title('Weighted Accuracy by Path Length')
plt.ylim(0, 1.0)
plt.xticks(rotation=45)
plt.tight_layout()

# Annotate bars with percentages
for bar, val in zip(bars, weighted_accuracies):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        val + 0.005,
        f"{val:.0%}",
        ha='center',
        va='bottom'
    )

plt.show()

In [ ]:
df = pd.read_csv("/home/cc/PHD/HealthBranches/questions_pro/final_dataset.csv")

In [ ]:
# 1) Carica il CSV con le domande da escludere
df_blocklist = pd.read_csv("/home/cc/PHD/HealthBranches/questions_to_remove.csv")

# 2) Carica il CSV originale che vuoi filtrare
df_orig = pd.read_csv("/home/cc/PHD/HealthBranches/questions_pro/dataset_updated_V2path.csv")

# 3) Filtra: tieni solo le righe il cui 'question' NON è nella blocklist
df_filtrato = df_orig[~df_orig['question'].isin(df_blocklist['question'])]

# 4) Scrivi il risultato su un nuovo CSV
df_filtrato.to_csv('tutte_le_domande_filtrate.csv', index=False)
print(len(df_filtrato))

In [ ]:
def is_valid_options(opt_str):
    """
    Ritorna True se opt_str può essere convertito in lista di lunghezza 5,
    altrimenti False (anche in caso di eccezione di parsing).
    """
    try:
        # Normalizza le virgolette e tenta l'ast.literal_eval
        opts = ast.literal_eval(
            opt_str
            .replace("['", '["')
            .replace("']", '"]')
            .replace("', '", '", "')
        )
        # Controlla che sia lista di lunghezza 5
        return isinstance(opts, list) and len(opts) == 5
    except (ValueError, SyntaxError):
        return False

# Applichiamo il filtro sul DataFrame originale
mask = df['options'].apply(is_valid_options)

# df filtrato: mantengo solo le righe valide
df = df.loc[mask].reset_index(drop=True)

In [ ]:
df.to_csv("/home/cc/PHD/HealthBranches/questions_pro/dataset_updated_V2path.csv", index=False)

In [ ]:
i = 0
for c in df['correct_option'].values:
    if c not in ["A", "B", "C", "D", "E"]:
        print(i)
        print(df.loc[i]['correct_option'])
    i = i+1

In [ ]:
df = pd.read_csv(f"{PATH}results/results_quiz_Llama-3.3-70B-Instruct-Turbo-Free.csv")
df["zero_shot_new"] = df["zero_shot"].apply(extract_option)

big_filtered = df[df["zero_shot_new"].str.upper() != df["real"].str.upper()]
len(big_filtered)

In [ ]:
small_df = pd.read_csv(f"{PATH}results/results_quiz_mistral.csv")
small_df = small_df[small_df["question"].isin(big_filtered["question"])]

In [ ]:
small_df["zero_shot_new"] = small_df["zero_shot"].apply(extract_option)
small_filtered = small_df[small_df["zero_shot_new"].str.upper() != small_df["real"].str.upper()]

In [ ]:
# Directory contenente i file CSV
csv_directory = "/home/cc/PHD/HealthBranches/quiz-ultimate-dataset"

# Leggere tutti i CSV
csv_files = [os.path.join(csv_directory, f) for f in os.listdir(csv_directory) if f.endswith(".csv")]
questions = []

print(csv_files)

# Elaborazione di ogni CSV
for file in csv_files:
    df = pd.read_csv(file)

    # Rimuove la colonna "zero_shot_rag" se esiste
    if "zero_shot_rag" in df.columns:
        df = df.drop(columns=["zero_shot_rag"])

    # Applica la funzione di estrazione su zero_shot
    df["zero_shot"] = df["zero_shot"].apply(extract_option)

    # Filtra le righe in cui zero_shot è diverso da real
    df_filtered = df[df["zero_shot"] != df["real"]].copy()

    questions.extend(df_filtered['question'].values)

In [ ]:
# Conta le occorrenze
conteggio = Counter(questions)

# Ordina in ordine decrescente
dizionario_ordinato = dict(sorted(conteggio.items(), key=lambda x: x[1], reverse=True))

filtered_dict = {k: v for k, v in dizionario_ordinato.items() if v >= 6}

print(len(filtered_dict))

In [ ]:
questions_to_check = pd.read_csv("/home/cc/PHD/HealthBranches/questions_to_check.csv")

# Trova le domande che sono sia nel dizionario che nel DataFrame
matching_questions = [q for q in filtered_dict.keys() if q in questions_to_check["question"].values]
print(len(matching_questions))

print("Domande trovate nel DataFrame:", len(matching_questions))

In [ ]:
# Carica il CSV (sostituisci 'file.csv' con il tuo file)
df = pd.read_csv("/home/cc/PHD/HealthBranches/results/results_checker_70B.csv")

# Conta le occorrenze di ogni valore nella colonna "check"
conteggio = df["check"].value_counts().sort_index()

# Stampa il risultato
print(conteggio)

In [ ]:
import pandas as pd
import ast
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

# Se la GPU è disponibile, usa 'cuda', altrimenti 'cpu'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Esecuzione su: {device}")

# Carica il modello sulla GPU se disponibile
model = SentenceTransformer('all-mpnet-base-v2', device=device)

# Sostituisci 'data.csv' con il percorso del tuo file CSV.
df = pd.read_csv('/home/cc/PHD/HealthBranches/questions_pro/dataset_updated_V2path.csv')

# Definisce la soglia di similarità (80%)
SIMILARITY_THRESHOLD = 0.9

# Pre-elabora il DataFrame:
# 1. Dividi il campo 'path' in una lista di segmenti
df['path_segments'] = df['path'].apply(lambda x: [seg.strip() for seg in x.split("->")])
# 2. Estrai il testo dell'opzione corretta
def extract_correct_option(row):
    try:
        options_list = ast.literal_eval(row['options'].replace("['", '["').replace("']", '"]').replace("', '", '", "'))
    except (ValueError, SyntaxError) as e:
        # Se il parsing fallisce, restituisce una stringa vuota
        return ""
    correct_letter = row['correct_option'].strip().upper()
    index = ord(correct_letter) - ord('A')
    if index < 0 or index >= len(options_list):
        return ""
    return options_list[index]

df['correct_option_text'] = df.apply(extract_correct_option, axis=1)

# Elabora in batch gli embedding delle opzioni corrette
correct_texts = df['correct_option_text'].tolist()
correct_embeddings = model.encode(correct_texts, device=device, convert_to_tensor=True)

# Funzione per verificare in una riga se almeno un segmento del path è semanticamente simile all'opzione corretta
def check_similar_option_batch(idx, row):
    segments = row['path_segments']
    # Se non ci sono segmenti, ritorna False
    if not segments:
        return False
    # Calcola in batch gli embedding dei segmenti (convert_to_tensor=True per operare su GPU)
    segments_embeddings = model.encode(segments, device=device, convert_to_tensor=True)
    # Recupera l'embedding dell'opzione corretta per la riga corrente
    correct_emb = correct_embeddings[idx]
    # Normalizza gli embedding per applicare correttamente il calcolo della similarità coseno
    correct_norm = correct_emb / correct_emb.norm()
    segments_norm = segments_embeddings / segments_embeddings.norm(dim=1, keepdim=True)
    # Calcola la similarità coseno tra l'opzione e ogni segmento
    similarities = torch.matmul(segments_norm, correct_norm)
    # Se almeno una similarità supera la soglia, ritorna True
    return (similarities >= SIMILARITY_THRESHOLD).any().item()

# Processa ogni riga e determina se il match semantico supera la soglia
results = [check_similar_option_batch(idx, row) for idx, row in tqdm(df.iterrows())]
df['is_similar_in_path'] = results

# Calcola e stampa il contatore finale
counter = sum(results)
print(f"Numero di righe in cui almeno un segmento del path risulta simile (>= 80%) all'opzione corretta: {counter}")

# Visualizza il DataFrame aggiornato
print(df)